In [ ]:
!git clone https://github.com/muxspace/facial_expressions

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 42.55 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Checking out files: 100% (13996/13996), done.


In [ ]:
import csv
data={}
with open('./facial_expressions/data/legend.csv') as f:
    reader=csv.reader(f)
    next(reader)#since the file has header to skip the first row next is use to put the pointer to the next line
    for row in reader:
        key=row[2].lower()#duto to uppercase and lower case same duplicate keys are getting stored
        if key in data:
            data[key].append(row[1])
        else:
            data[key]=[row[1]]
    

In [ ]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('main_data')
os.mkdir('main_data/training')
os.mkdir('main_data/testing')

In [ ]:
for emotion in emotion_list:#for adding each expression's directories  
    os.mkdir(os.path.join('main_data/training/',emotion))
    os.mkdir(os.path.join('main_data/testing/',emotion))

In [ ]:
#filing the directories of emotions using the images directory
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
    train_size=int(split_size*len(images))
    train_images=images[:train_size]
    test_images=images[train_size:]
    for images in train_images:
        source=os.path.join('./facial_expressions/images',images)
        dest=os.path.join('./main_data/training',emotion,images)
        copyfile(source,dest)
    for images in test_images:
        source=os.path.join('./facial_expressions/images',images)
        dest=os.path.join('./main_data/testing',emotion,images)
        copyfile(source,dest)

In [ ]:
#ImageDataGenerator class use to first convert the image in to array and then one-hot encode it
#also we can flip rotate zoom in zoom out
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(8,activation='softmax')
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 16)        448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 16)        4624      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)             

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#training the model
train_dir='./main_data/training'
test_dir='./main_data/testing'
train_gen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_gen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=128)
test_gen=ImageDataGenerator(rescale=1.0/255)
test_generator=train_gen.flow_from_directory(test_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)#used to stop the epoch when the model is not improving


In [ ]:
model.fit_generator(train_generator,
                    epochs = 10,
                    verbose = 1,
                    validation_data = test_generator,
                    callbacks = [es])
#verbose=1 is used to print the data of training

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
86/86 [==============================] - 15s 175ms/step - loss: 1.4847 - acc: 0.4796 - val_loss: 1.0255 - val_acc: 0.5492
Epoch 2/10
86/86 [==============================] - 15s 170ms/step - loss: 0.9371 - acc: 0.6357 - val_loss: 0.9828 - val_acc: 0.6298
Epoch 3/10
86/86 [==============================] - 14s 169ms/step - loss: 0.7832 - acc: 0.7281 - val_loss: 0.9334 - val_acc: 0.6444
Epoch 4/10
86/86 [==============================] - 15s 171ms/step - loss: 0.7508 - acc: 0.7378 - val_loss: 0.9230 - val_acc: 0.6579
Epoch 5/10
86/86 [==============================] - 15s 171ms/step - loss: 0.7327 - acc: 0.7455 - val_loss: 0.9655 - val_acc: 0.6466
Epoch 6/10
86/86 [==============================] - 15s 172ms/step - loss: 0.7031 - acc: 0.7548 - val_loss: 0.9812 - val_acc: 0.6430


In [19]:
for i in model.predict_classes(test_generator):
  print(i,end=' ')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


5 4 4 5 4 5 2 5 5 2 4 5 4 5 2 5 4 5 4 5 4 5 5 4 4 5 5 5 4 5 5 2 5 5 5 5 5 4 5 5 4 5 4 5 5 5 4 4 5 4 5 5 5 5 4 4 5 4 4 4 5 4 5 5 5 4 5 2 5 4 5 5 5 4 5 4 2 4 5 4 4 5 5 5 5 2 5 5 5 5 4 6 5 4 2 5 4 4 4 5 5 4 5 5 5 5 4 5 5 4 5 5 5 4 5 4 5 5 5 5 4 5 4 5 5 5 4 5 5 5 5 4 5 5 5 5 5 5 4 4 4 4 5 4 5 5 5 4 5 5 5 5 5 5 2 5 2 5 5 5 5 4 4 5 5 5 4 2 5 4 5 4 5 5 2 5 2 5 5 4 4 5 2 5 5 5 4 5 5 5 5 6 4 5 5 5 4 4 2 5 4 5 5 5 5 5 5 5 5 2 5 4 5 4 5 4 5 5 5 4 5 4 5 5 4 5 2 5 4 4 5 5 5 5 5 4 4 4 5 4 5 4 5 5 5 5 5 5 4 5 4 5 5 5 5 4 5 4 5 5 5 4 5 5 4 5 5 5 4 5 5 5 4 5 5 4 5 5 5 2 4 4 2 4 5 5 5 4 5 5 5 5 5 5 5 4 5 5 5 5 4 5 4 5 4 5 5 5 5 2 4 5 4 4 6 4 5 4 5 5 5 4 4 5 4 4 5 4 4 5 4 4 4 2 5 5 5 4 5 4 4 5 5 4 5 5 4 5 4 4 5 4 5 5 5 4 5 5 2 5 5 5 5 4 4 5 4 5 5 5 5 2 4 5 4 5 5 4 4 4 5 4 5 5 2 4 5 5 4 5 4 5 4 5 5 2 5 5 5 4 5 4 5 2 5 5 4 5 5 4 5 2 5 4 5 4 5 6 4 4 5 5 5 2 4 2 5 2 4 5 4 4 5 5 5 5 5 5 5 5 2 5 5 5 5 4 5 5 4 5 5 5 2 2 4 2 5 5 5 5 5 4 6 4 5 4 5 5 5 4 4 5 2 2 4 5 5 4 2 4 4 4 4 5 4 5 4 5 5 5 4 5 5 5 5 5 5 5 5 4 